In [123]:
import numpy as np
import pandas as pd

In [ ]:
'''
22.12. 기준
인구소멸 위기지역 (기초자치단체(시,군) / 특례시(구):수원,고양,용인 행정구(구): 성남,안산,안양) top10
가임기 여성인구(20-39) / 60세이상 노령인구
'''

In [124]:
pe = pd.read_csv('data/주민등록인구집계현황.csv', encoding='euc-kr', thousands=',')
pe.head(3)

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2022,12,도,경기도,13589432,1040954,1327726,1737291,1882300,2284603,...,647283,821150,904073,1121889,1133428,878494,437251,252711,44184,1369
1,2022,12,시군,경기도 가평군,62197,2930,4463,5929,5397,7589,...,2186,2503,2433,3525,5090,6439,3653,2408,411,13
2,2022,12,읍면동,경기도 가평군 가평읍,19555,1020,1673,2094,1795,2630,...,778,937,848,1244,1583,1805,1054,739,126,6


In [125]:
pe.isna().sum()

연도             0
월              0
행정구역구분명        0
행정구역명          0
총 인구수          0
0~9세           0
10~19세         0
20~29세         0
30~39세         0
40~49세         0
50~59세         0
60~69세         0
70~79세         0
80~89세         0
90~99세         0
100세 이상        0
총 인구수 (남)      0
0~9세 (남)       0
10~19세 (남)     0
20~29세 (남)     0
30~39세 (남)     0
40~49세 (남)     0
50~59세 (남)     0
60~69세 (남)     0
70~79세 (남)     0
80~89세 (남)     0
90~99세 (남)     0
100세 이상 (남)    0
총 인구수 (여)      0
0~9세 (여)       0
10~19세 (여)     0
20~29세 (여)     0
30~39세 (여)     0
40~49세 (여)     0
50~59세 (여)     0
60~69세 (여)     0
70~79세 (여)     0
80~89세 (여)     0
90~99세 (여)     0
100세 이상 (여)    0
dtype: int64

In [126]:
pe.columns

Index(['연도', '월', '행정구역구분명', '행정구역명', '총 인구수', '0~9세', '10~19세', '20~29세',
       '30~39세', '40~49세', '50~59세', '60~69세', '70~79세', '80~89세', '90~99세',
       '100세 이상', '총 인구수 (남)', '0~9세 (남)', '10~19세 (남)', '20~29세 (남)',
       '30~39세 (남)', '40~49세 (남)', '50~59세 (남)', '60~69세 (남)', '70~79세 (남)',
       '80~89세 (남)', '90~99세 (남)', '100세 이상 (남)', '총 인구수 (여)', '0~9세 (여)',
       '10~19세 (여)', '20~29세 (여)', '30~39세 (여)', '40~49세 (여)', '50~59세 (여)',
       '60~69세 (여)', '70~79세 (여)', '80~89세 (여)', '90~99세 (여)', '100세 이상 (여)'],
      dtype='object')

In [127]:
# 인구소멸위기지역 column만들기
pe['인구소멸위기지역'] = ((pe['20~29세 (여)']+pe['30~39세 (여)']) / (pe['60~69세']+pe['70~79세']+pe['80~89세']+pe['90~99세']+pe['100세 이상'])).round(2)

In [128]:
# dataframe 다시 만들기
pe = pe[['행정구역구분명','행정구역명','인구소멸위기지역']]
pe

,행정구역구분명,행정구역명,인구소멸위기지역
0,도,경기도,0.57
1,시군,경기도 가평군,0.20
2,읍면동,경기도 가평군 가평읍,0.26
3,읍면동,경기도 가평군 북면,0.09
4,읍면동,경기도 가평군 상면,0.11
...,...,...,...
615,읍면동,경기도 화성시 팔탄면,0.16
616,읍면동,경기도 화성시 향남읍,0.86
617,읍면동,경기도 화성시 화산동,0.47
618,읍면동,경기도 화성시동부출장소,NaN


In [129]:
pe.isna().sum()

행정구역구분명     0
행정구역명       0
인구소멸위기지역    5
dtype: int64

In [130]:
pe[pe['행정구역구분명'].str.contains('읍면동')]

,행정구역구분명,행정구역명,인구소멸위기지역
2,읍면동,경기도 가평군 가평읍,0.26
3,읍면동,경기도 가평군 북면,0.09
4,읍면동,경기도 가평군 상면,0.11
5,읍면동,경기도 가평군 설악면,0.18
6,읍면동,경기도 가평군 조종면,0.26
...,...,...,...
615,읍면동,경기도 화성시 팔탄면,0.16
616,읍면동,경기도 화성시 향남읍,0.86
617,읍면동,경기도 화성시 화산동,0.47
618,읍면동,경기도 화성시동부출장소,NaN


In [131]:
# 읍면동 삭제
pe= pe.drop(pe[pe['행정구역구분명'] == '읍면동'].index)
pe.head()

,행정구역구분명,행정구역명,인구소멸위기지역
0,도,경기도,0.57
1,시군,경기도 가평군,0.20
8,시군,경기도 고양시,0.57
9,구,경기도 고양시 덕양구,0.56
31,구,경기도 고양시 일산동구,0.60


In [132]:
pe.index

Int64Index([  0,   1,   8,   9,  31,  44,  56,  63,  83,  96, 100, 109, 122,
            137, 156, 165, 176, 177, 200, 218, 231, 232, 245, 258, 269, 280,
            300, 301, 314, 328, 344, 345, 363, 391, 404, 417, 428, 435, 436,
            452, 464, 477, 484, 499, 514, 532, 558, 574, 589],
           dtype='int64')

In [133]:
pe['행정구역명'][231]

'경기도 수원시 '

In [134]:
# 특례시,행정구 삭제
pe = pe.drop(pe[pe['행정구역명'] == '경기도 수원시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 고양시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 용인시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 성남시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 안산시 '].index)
pe = pe.drop(pe[pe['행정구역명'] == '경기도 안양시 '].index)

In [135]:
pe

,행정구역구분명,행정구역명,인구소멸위기지역
0,도,경기도,0.57
1,시군,경기도 가평군,0.20
9,구,경기도 고양시 덕양구,0.56
31,구,경기도 고양시 일산동구,0.60
44,구,경기도 고양시 일산서구,0.56
56,시군,경기도 과천시,0.62
63,시군,경기도 광명시,0.53
83,시군,경기도 광주시,0.51
96,시군,경기도 광주시 오포2동,0.60
100,시군,경기도 구리시,0.53


In [136]:
# 경기도 삭제
pe.drop([0], inplace=True) 
pe.head()

,행정구역구분명,행정구역명,인구소멸위기지역
1,시군,경기도 가평군,0.20
9,구,경기도 고양시 덕양구,0.56
31,구,경기도 고양시 일산동구,0.60
44,구,경기도 고양시 일산서구,0.56
56,시군,경기도 과천시,0.62


In [137]:
pe.isna().sum()

행정구역구분명     0
행정구역명       0
인구소멸위기지역    0
dtype: int64

In [138]:
pe

,행정구역구분명,행정구역명,인구소멸위기지역
1,시군,경기도 가평군,0.20
9,구,경기도 고양시 덕양구,0.56
31,구,경기도 고양시 일산동구,0.60
44,구,경기도 고양시 일산서구,0.56
56,시군,경기도 과천시,0.62
63,시군,경기도 광명시,0.53
83,시군,경기도 광주시,0.51
96,시군,경기도 광주시 오포2동,0.60
100,시군,경기도 구리시,0.53
109,시군,경기도 군포시,0.54


In [139]:
# 0.5이하 인구소멸위기지역
# 인구소멸top10
sorted_pe = pe.sort_values('인구소멸위기지역')
sorted_pe.head(10)

,행정구역구분명,행정구역명,인구소멸위기지역
1,시군,경기도 가평군,0.20
417,시군,경기도 연천군,0.20
391,시군,경기도 양평군,0.20
404,시군,경기도 여주시,0.27
558,시군,경기도 포천시,0.28
156,시군,경기도 동두천시,0.33
328,시군,경기도 안성시,0.38
464,구,경기도 용인시 처인구,0.48
363,구,경기도 안양시 만안구,0.48
218,구,경기도 성남시 중원구,0.50


In [140]:
# 인구많은 top10
sorted_pe2 = sorted_pe.sort_values('인구소멸위기지역', ascending=False)
sorted_pe2.head(10)

,행정구역구분명,행정구역명,인구소멸위기지역
589,시군,경기도 화성시,0.87
428,시군,경기도 오산시,0.76
280,시군,경기도 시흥시,0.72
574,시군,경기도 하남시,0.67
452,구,경기도 용인시 수지구,0.66
177,구,경기도 성남시 분당구,0.66
532,시군,경기도 평택시,0.65
301,구,경기도 안산시 단원구,0.64
345,구,경기도 안양시 동안구,0.63
232,구,경기도 수원시 권선구,0.63
